<a href="https://colab.research.google.com/github/Addaci/fine-tuned-mt5/blob/main/HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install --upgrade transformers datasets huggingface_hub gradio

# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict

# Define file paths
csv_path = "/content/drive/MyDrive/Colab Notebooks/HCA_1358_Paired_Line_Text_Ver.1.2_15102024.csv"

# Load the CSV data into a Pandas DataFrame
df = pd.read_csv(csv_path)  # Pandas automatically detects the header

# Get the raw_htr and groundtruth lists from the DataFrame
raw_htr_texts = df['Raw-HTR Text'].tolist()  # Accessing by column name
groundtruth_texts = df['Hand-corrected Groundtruth'].tolist()

# Initialize the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

# Tokenize the data
def preprocess_function(examples):
    inputs = [doc for doc in examples['Raw-HTR Text']]  # Using column name
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        # Corrected line: removed the line break within the string
        labels = tokenizer(examples['Hand-corrected Groundtruth'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create a Hugging Face Dataset
data = Dataset.from_dict({"Raw-HTR Text": raw_htr_texts, "Hand-corrected Groundtruth": groundtruth_texts})  # Using column names
tokenized_data = data.map(preprocess_function, batched=True)

# Split the data into training and validation sets
train_testvalid = tokenized_data.train_test_split(test_size=0.2, seed=42) # Seed added for consistent split
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42) # Seed added for consistent split
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']
})

# Initialize the mT5-small model
model = T5ForConditionalGeneration.from_pretrained("google/mt5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_experiment_4",
    per_device_train_batch_size=8,  # Adjust based on your GPU memory
    per_device_eval_batch_size=8,
    # predict_with_generate=True,  # Removed in previous step
    learning_rate=2e-5,  # You can experiment with different learning rates
    num_train_epochs=10,  # Adjusted to 10 epochs
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # Early stopping parameters:
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    # early_stopping_patience=3, # Removed for now
    logging_strategy="epoch",  # Log training loss at each epoch
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"]
)

# Fine-tune the model
trainer.train()


# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 514.6 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,50.653500,43.558308
